# Pyglet

In [1]:
import pyglet, time
import numpy as np
import matplotlib.pyplot as plt
from pyglet.gl import *
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)

%cd /Users/e341194/Documents/GitHub/Islands/
%run worldgen.py
%matplotlib inline

C:\Users\e341194\Documents\GitHub\Islands


# IPython Integration

Per the 3.x
[documentation](http://ipython.org/ipython-doc/stable/config/eventloops.html)
to integrate an event loop with the kernel (not the terminal) it must occasionally call
[`IPython.kernel.zmq.kernelbase.Kernel.do_one_iteration()`](http://ipython.org/ipython-doc/stable/api/generated/IPython.kernel.zmq.kernelbase.html#IPython.kernel.zmq.kernelbase.Kernel.do_one_iteration).

The decorator `@register_integration(<name>)` will add special semantics to the `%gui <name>` magic.

Wonder if scheduling calls through `pyglet.clock.schedule` will work?

In [ ]:
from IPython.kernel.zmq.eventloops import register_integration

def kernel_callback(dt, kernel):
    kernel.do_one_iteration()

@register_integration('pyglet')
def pyglet_event_loop(kernel):
    pyglet.clock.schedule_interval(kernel_callback, 0.1, kernel)
    pyglet.app.run()

In [ ]:
%gui pyglet

Yes, yes it does. One thing that doesn't seem to work so well, however, is
stopping and restarting the pyglet event loop. As in, doesn't work at all.
Another issue seems to be CPU resource issue. The callback interval could
probably be tweaked to trade off response time vs. cpu usage.

Okay, now can we tweak event handlers while the app is live?

# Graphics

In [4]:
world_width = 80
world_height = 50 # n x n grid
imsize = 12 # Font is 12x12

worldpx_w, worldpx_h = world_width * imsize, world_height * imsize

terrain = np.zeros((world_height, world_width))
im_set = pyglet.image.load('12x12.png')
im_set_seq  = pyglet.image.ImageGrid(im_set, 48, 16)
im_set_tex_seq = pyglet.image.TextureGrid(im_set_seq)    


In [5]:
plt.imshow(terrain)

In [ ]:
canvas = pyglet.window.Window(worldpx_w, worldpx_h)


Draw a glyph.

In [7]:
glyph0 = pyglet.sprite.Sprite(im_set_seq[0])
glyph1 = pyglet.sprite.Sprite(im_set_seq[0, 1])

glyph0.x, glyph0.y = canvas.width//2, canvas.height//2

In [30]:
water = pyglet.image.SolidColorImagePattern(color_rgba(keyrgb_color_int[0]))
white = pyglet.image.SolidColorImagePattern((0, 0, 50, 255))
water_square = pyglet.sprite.Sprite(water.create_image(imsize, imsize))
big_white = pyglet.sprite.Sprite(white.create_image(12, 12))

In [29]:
glyph1.x, glyph1.y = 10, 10
@canvas.event
def on_draw():
    canvas.clear()
    glyph0.draw()
    water_square.draw()
    big_white.draw()
    glyph1.draw()


In [31]:
big_white.x, big_white.y = 10, 10

In [ ]:
glyph0.color = (0,0,50)

In [41]:
glyph1.opacity = 255

In [40]:
def update(dt):
    # Move 10 pixels per second
    if glyph1.x > canvas.width:
        glyph1.x = 0
    else:
        glyph1.x += dt * 10

# Call update 60 times a second
pyglet.clock.schedule_interval(update, 1/60.)

In [43]:
dir(glyph1)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_animate',
 '_animation',
 '_batch',
 '_create_vertex_list',
 '_event_stack',
 '_get_batch',
 '_get_group',
 '_get_handlers',
 '_get_height',
 '_get_image',
 '_get_width',
 '_group',
 '_opacity',
 '_raise_dispatch_exception',
 '_rgb',
 '_rotation',
 '_scale',
 '_set_batch',
 '_set_color',
 '_set_group',
 '_set_image',
 '_set_opacity',
 '_set_rotation',
 '_set_scale',
 '_set_texture',
 '_set_visible',
 '_set_x',
 '_set_y',
 '_subpixel',
 '_texture',
 '_update_color',
 '_update_position',
 '_usage',
 '_vertex_list',
 '_visible',
 '_x',
 '_y',
 'batch',
 'color',
 'delete',
 'dispatch_event',
 'draw',
 'event',
 'event_types',
 'grou